In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  with open('/content/drive/MyDrive/ML/Course recommendation system/' + fn, 'wb') as f:
    f.write(uploaded[fn])

Saving udemy_course_data.csv to udemy_course_data (1).csv
User uploaded file "udemy_course_data (1).csv" with length 842660 bytes


In [3]:
!pip install neattext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 3.7 MB/s eta 0:00:00


In [4]:
# Necessary imports
import pandas as pd
import numpy as np
import neattext.functions as nfx
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/ML/Course recommendation system/udemy_course_data.csv')

In [13]:
# Display basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   course_id            3683 non-null   int64 
 1   course_title         3683 non-null   object
 2   url                  3683 non-null   object
 3   is_paid              3683 non-null   bool  
 4   price                3683 non-null   int64 
 5   num_subscribers      3683 non-null   int64 
 6   num_reviews          3683 non-null   int64 
 7   num_lectures         3683 non-null   int64 
 8   level                3683 non-null   object
 9   content_duration     3683 non-null   object
 10  published_timestamp  3683 non-null   object
 11  subject              3683 non-null   object
 12  profit               3683 non-null   int64 
 13  published_date       3683 non-null   object
 14  published_time       3682 non-null   object
 15  year                 3683 non-null   int64 
 16  month 

In [12]:
# Display the first few rows of the dataset
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance,97830,2016-12-19,19:26:30Z,2016,12,19
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance,232845,2017-05-30,20:07:24Z,2017,5,30
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance,255200,2016-12-13,14:57:18Z,2016,12,13


In [14]:
# EDA: Descriptive statistics
df.describe()

,course_id,price,num_subscribers,num_reviews,num_lectures,profit,year,month,day
count,3.683000e+03,3683.000000,3683.000000,3683.000000,3683.000000,3.683000e+03,3683.000000,3683.000000,3683.000000
mean,6.764546e+05,65.992398,3193.371165,156.448004,40.062178,2.402885e+05,2015.433342,6.162639,15.841162
std,3.437217e+05,60.985586,9498.231406,935.078241,50.366788,1.000760e+06,1.185920,3.379314,8.780906
min,8.324000e+03,0.000000,0.000000,0.000000,0.000000,0.000000e+00,2011.000000,1.000000,1.000000
25%,4.077270e+05,20.000000,110.000000,4.000000,15.000000,1.567500e+03,2015.000000,3.000000,8.000000
50%,6.882440e+05,45.000000,911.000000,18.000000,25.000000,2.305000e+04,2016.000000,6.000000,16.000000
75%,9.617290e+05,95.000000,2537.500000,67.000000,45.000000,1.182600e+05,2016.000000,9.000000,23.000000
max,1.282064e+06,200.000000,268923.000000,27445.000000,779.000000,2.431680e+07,2017.000000,12.000000,31.000000


In [15]:
# EDA: Check for missing values
df.isnull().sum()

,0
course_id,0
course_title,0
url,0
is_paid,0
price,0
num_subscribers,0
num_reviews,0
num_lectures,0
level,0
content_duration,0


In [23]:
# Preprocess the data
df['Clean_title'] = df['course_title'].apply(nfx.remove_stopwords)
df['Clean_title'] = df['Clean_title'].apply(nfx.remove_special_characters)

In [24]:
# Create a CountVectorizer object
count_vect = CountVectorizer()
cv_mat = count_vect.fit_transform(df['Clean_title'])

In [25]:
# Calculate cosine similarity
cosine_sim_mat = cosine_similarity(cv_mat)

In [28]:
# Create a Series with course titles as index and their respective index as values
course_indices = pd.Series(df.index, index=df['course_title']).drop_duplicates()

In [31]:
def recommend_courses(title, num_recommendations=10, course_indices=course_indices):
    """
    Recommend courses based on the given course title.

    Args:
    title (str): The title of the course to base recommendations on.
    num_recommendations (int): The number of recommendations to return.

    Returns:
    pd.DataFrame: A dataframe of recommended courses with their details.
    """
    # Get the index of the course that matches the title
    idx = course_indices[title]

    # Get the pairwise similarity scores of all courses with that course
    sim_scores = list(enumerate(cosine_sim_mat[idx]))

    # Sort the courses based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar courses
    sim_scores = sim_scores[1:num_recommendations+1]

    # Get the course indices
    course_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar courses
    recommended_courses = df.iloc[course_indices][['course_title', 'url', 'price', 'num_subscribers']]
    recommended_courses['similarity_score'] = [i[1] for i in sim_scores]

    return recommended_courses

In [36]:
# Example usage
sample_course = "Trading Options Basics"
recommendations = recommend_courses(sample_course, 5)
print(f"Recommendations for '{sample_course}':")
recommendations

Recommendations for 'Trading Options Basics':


,course_title,url,price,num_subscribers,similarity_score
97,Options Trading 101: The Basics,https://www.udemy.com/options-trading-101-the-...,0,1514,0.866025
195,Trading Options For Consistent Returns: Option...,https://www.udemy.com/trading-options-for-income/,0,4077,0.816497
864,Basics of Trading,https://www.udemy.com/basicsoftrading/,40,99,0.816497
66,Options Trading Basics (3-Course Bundle),https://www.udemy.com/learn-options-trading-co...,180,10100,0.774597
802,Trading: Basics of Trading for Beginners,https://www.udemy.com/money-basics-of-trading-...,140,1309,0.707107


In [33]:
# Export the processed dataframe to CSV
df.to_csv('/content/drive/MyDrive/ML/Course recommendation system/UdemyCleanedTitle.csv', index=None)
print("Processed data exported to 'UdemyCleanedTitle.csv'")

Processed data exported to 'UdemyCleanedTitle.csv'
